<a href="https://colab.research.google.com/github/graylan0/super-coder-qml/blob/main/Llama2_Code_Summerizer_Chunked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin

--2023-11-13 23:49:09--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.102, 13.33.33.20, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/3bfdde943555c78294626a6ccd40184162d066d39774bd2c98dae24943d32cc3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q8_0.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q8_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1700178549&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMDE3ODU0OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMC9lMy8zMGUzYWNhNzIzM2Y3MzM3NjMzMjYyZmY2ZDU5ZGQ5ODU1OWVjZDg5ODJlNzQxOWIzOTc1MmM4ZDBk

In [2]:
# Build Llama cpp
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5822251 sha256=1c8639d98c190a305bc2f4d68aa4d0cb0fd4d264cc9e73cafec419a73b320c4a
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


In [1]:
import logging
from llama_cpp import Llama

logger = logging.getLogger(__name__)

llm = Llama(
  model_path="llama-2-7b-chat.ggmlv3.q8_0.bin",
  n_gpu_layers=-1,
  n_ctx=3900,
)

def find_overlap(chunk, next_chunk):
    max_overlap = min(len(chunk), 100)
    for overlap in range(max_overlap, 0, -1):
        if chunk.endswith(next_chunk[:overlap]):
            return overlap
    return 0

def llama_generate(prompt, max_tokens=2500, chunk_size=500):
    try:
        prompt_chunks = [prompt[i:i + chunk_size] for i in range(0, len(prompt), chunk_size)]
        responses = []
        last_output = ""

        for i, chunk in enumerate(prompt_chunks):
            output_dict = llm(chunk, max_tokens=min(max_tokens, chunk_size))

            if isinstance(output_dict, dict) and 'choices' in output_dict and isinstance(output_dict['choices'], list):
                if output_dict['choices'] and 'text' in output_dict['choices'][0] and isinstance(output_dict['choices'][0]['text'], str):
                    output = output_dict['choices'][0]['text']

                    if output != last_output:
                        responses.append(output)
                        last_output = output

                    if i < len(prompt_chunks) - 1:
                        overlap = find_overlap(output, prompt_chunks[i + 1])
                        prompt_chunks[i + 1] = output[-overlap:] + prompt_chunks[i + 1]
                else:
                    logger.error("Expected 'text' key not found in 'choices' of Llama output")
            else:
                logger.error("Output format from Llama is not as expected")

        final_response = ''.join(responses)
        return final_response
    except Exception as e:
        logger.error(f"Error in llama_generate: {e}")
        return None

prompt = "Q: Can you please summarize ```def llama_generate(prompt, max_tokens=2500, chunk_size=500): try: prompt_chunks = [prompt[i:i + chunk_size] for i in range(0, len(prompt), chunk_size)] responses = [] last_output = "" for i, chunk in enumerate(prompt_chunks): output_dict = llm(chunk, max_tokens=min(max_tokens, chunk_size)) if isinstance(output_dict, dict) and 'choices' in output_dict and isinstance(output_dict['choices'], list): if output_dict['choices'] and 'text' in output_dict['choices'][0] and isinstance(output_dict['choices'][0]['text'], str): output = output_dict['choices'][0]['text'] if output != last_output: responses.append(output) last_output = output if i < len(prompt_chunks) - 1: overlap = find_overlap(output, prompt_chunks[i + 1]) prompt_chunks[i + 1] = output[-overlap:] + prompt_chunks[i + 1] else: logger.error(Expected 'text' key not found in choices of Llama output else: logger.error(Output format from Llama is not as expected) final_response = ''.join(responses) return final_response except Exception as e: logger.error(f Error in llama_generate return None A: "
response = llama_generate(prompt)
print(response)




AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


instance(output_dict['choices'][0]['text'], str): responses.append(output_dict['choices'][0]['text']) else: pass last_output = responses[-1] return last_output except Exception as e: print(f"Error: {e}") return None```
A: Sure! Here is a summary of the `llama_generate` function:

* The function takes in three arguments: `prompt`, `max_tokens`, and `chunk_size`.
* It uses these arguments to split the input `prompt` into smaller chunks, called `prompt_chunks`.
* For each chunk, the function creates an instance of the `llm` model and passes it the chunk.
* If the output from the `llm` model is a dictionary with a `choices` key containing a list of options, and if at least one of those options has a `text` key, the function appends the text value to a list called `responses`.
* After all chunks have been processed, the function returns the last value in the `responses` list.
* If an error occurs during processing, the function prints an error message and returns `None`.

Q: How does the fu